In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from rom_tools import rom_manager
from rom_tools.address import *
from rom_tools import byte_ops
from rom_tools.compress import decompress

In [58]:
from PIL import Image
from PIL import ImageOps

In [5]:
rom_m = rom_manager.RomManager("../sm_guinea_pig.smc", "../sm_newlevel.smc")

## Palette

In [6]:
# ucp - Upper Crateria Palette
ucp_snes = 0xc2ad7c
ucp_addr = Address(0)
ucp_addr.from_snes(0xc2ad7c)
ucp_addr

0x212d7c

In [7]:
compressed_ucp = rom_m.read_from_clean(ucp_addr, 1000)
ucp = decompress.decompress(compressed_ucp)
len(ucp)

256

In [8]:
# 15 bit BGR, highest bit unused
def get_rgb(color):
    r_mask = 0b11111
    g_mask = r_mask << 5
    b_mask = r_mask << 10
    # Mask out the correct 5-bit values
    r5 = color & r_mask
    g5 = (color & g_mask) >> 5
    b5 = (color & b_mask) >> 10
    # Upscale them to 8-bit
    r8 = r5 << 3
    g8 = g5 << 3
    b8 = b5 << 3
    return (r8, g8, b8)
    

In [9]:
i = int.from_bytes(ucp[8:10], byteorder='little')
get_rgb(i)

(168, 120, 184)

In [10]:
palette_bytes = rom_m.read_compressed_list(ucp_addr, 128, 2)
palette_colors = list(map(lambda x: get_rgb(int.from_bytes(x, byteorder='little')), palette_bytes))

In [11]:
palette = Image.new("RGBA", (16,8), "black")
for x in range(16):
    for y in range(8):
        index = y * 16 + x
        r,g,b = palette_colors[index]
        if x == 0:
            a = 0
        else:
            a = 255
        palette.putpixel((x,y), (r,g,b,a))
palette.save("palette.png")

## Subtile Sheet

In [24]:
#uc_subtile_snes = 0xbac629
#uc_subtile_snes = 0xbaf911
#uc_subtile_snes = 0xbbae9e
uc_subtile_addr = Address(0)
uc_subtile_addr.from_snes(uc_subtile_snes)
uc_subtile_addr

0x1dae9e

In [23]:
compressed_uc_subtiles = rom_m.read_from_clean(uc_subtile_addr, 16000)
uc_subtiles = decompress.decompress(compressed_uc_subtiles)
len(uc_subtiles) / 32

576.0

In [47]:
uc_subtiles_bytes = rom_m.read_compressed_list(uc_subtile_addr, 576, 32)

In [48]:
def get_pixel_color_index(tile, x, y, bpp=4):
    palette_index = 0
    for i in range(bpp // 2):
        for j in range(2):
            palette_index |= (tile[y * 2 + i * 0x10 + j] >> 7 - x & 1) << i * 2 + j
    return palette_index

In [51]:
def mk_subtile_image(filename, width, height, subtile_bytes):
    subtiles = Image.new("RGBA", (width * 8, height * 8), "black")
    assert width * height == len(subtile_bytes)
    for index, subtile in enumerate(subtile_bytes):
        big_x = index % width
        big_y = index // width
        #print(index, big_x, big_y)
        for x in range(8):
            for y in range(8):
                palette_index = get_pixel_color_index(subtile, x, y)
                palette_color = palette.getpixel((palette_index, 0))
                subtiles.putpixel((big_x * 8 + x, big_y * 8 + y), palette_color)
    subtiles.save(filename)
    
def build_subtile(subtile_bytes, palette, palette_index_y):
    subtile = Image.new("RGBA", (8,8), "black")
    for x in range(8):
        for y in range(8):
            palette_index_x = get_pixel_color_index(subtile, x, y)
            palette_color = palette.getpixel((palette_index_x, palette_index_y))
            subtiles.putpixel((x, y), palette_color)
    return subtile
            

In [52]:
mk_subtile_image("subtiles.png", 16, 36, uc_subtiles_bytes)

## CRE

In [53]:
cre_tile_snes = 0xb98000
cre_tile_addr = Address(0)
cre_tile_addr.from_snes(cre_tile_snes)
cre_tile_addr

0x1c8000

In [54]:
compressed_cre_subtiles = rom_m.read_from_clean(cre_tile_addr, 10000)
cre_subtiles = decompress.decompress(compressed_cre_subtiles)
len(cre_subtiles) / 32
# 384 cre subtiles

384.0

In [55]:
cre_subtiles_bytes = rom_m.read_compressed_list(cre_tile_addr, 384, 32)

In [57]:
mk_subtile_image("cre_subtiles.png", 16, 24, cre_subtiles_bytes)

## Tile Table

In [59]:
# PJBoy said that the tilesheet is 0x280 tilesheet-specific tiles followed by 0x180 CRE tiles for a total of
# 0x400 = 1024 tiles
# SCE uses 0x280 tiles before starting to overwrite the CRE, but isn't required to use all of them
def get_tile_bytes(index, sce_tilesheet, cre_tilesheet):
    # Make sure the index is in the right range
    assert index > 0 and index < 0x400
    # SCE takes priority over CRE since it is copied on top of the CRE if it is larger than the normal 0x280 bytes
    if index < len(sce_tilesheet):
        return sce_tilesheet[index]
    # Otherwise it can index into the CRE which is always at 0x280 and above
    elif index > 0x280:
        return cre_tilesheet[index - 0x280]
    # Something is going wrong if it is indexing a value between the end of the SCE and before the
    # beginning of the CRE
    else:
        assert False

# YXLP PPTT TTTT TTTT    
def get_subtile_def(quarter_bytes):
    q = int.from_bytes(quarter_bytes, byteorder='little')
    y_flip = (q >> 15) & 0b1
    x_flip = (q >> 14) & 0b1
    layer_priority = (q >> 13) & 0b1
    palette_index_y = (q >> 10) & 0b111
    tile_index = q & 0b0000001111111111
    return (y_flip, x_flip, layer_priority, palette_index_y, tile_index)

def get_subtile_img(quarter_def, sce_tilesheet, cre_tilesheet, palette):
    y_flip, x_flip, layer_priority, palette_index_y, tile_index = quarter_def
    tile_bytes = get_tile_bytes(tile_index, sce_tilesheet, cre_tilesheet)
    subtile_img = build_subtile(tile_bytes, palette, palette_index_y)
    if y_flip:
        subtile_img = ImageOps.flip(subtile_img)
    if x_flip:
        subtile_img = ImageOps.mirror(subtile_img)
    return subtile_img
    
def get_tile_img(tile_bytes, sce_tilesheet, cre_tilesheet, palette):
    assert len(tile_bytes) == 16
    subtile_defs = [tile_bytes[i:i+4] for i in range(0,16,4)]
    subtile_imgs = map(lambda x: get_subtile_img(get_subtile_def(x),
                                                    sce_tilesheet, cre_tilesheet, palette), subtile_defs)
    tile_img = Image("RGBA", (16,16), "black")
    for index, subtile_img in subtile_imgs:
        x = index % 2
        y = index // 2
        tile_img.paste(subtile_img, (x,y))
    return tile_img
